# 1. Loading Tags to RDS

In [2]:
import time
import json
import datetime
import random
import math
import configparser

import boto3
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from tqdm import tqdm

In [3]:
ssm = boto3.client('ssm')
dbhost = ssm.get_parameter(Name='/moviestream/dbhost', WithDecryption=True)['Parameter']['Value']
dbuser = ssm.get_parameter(Name='/moviestream/dbuser', WithDecryption=True)['Parameter']['Value']
dbpass = ssm.get_parameter(Name='/moviestream/dbpass', WithDecryption=True)['Parameter']['Value']
dbname = ssm.get_parameter(Name='/moviestream/dbname', WithDecryption=True)['Parameter']['Value']
data_folder = 'app-data'
file = 'tags.csv'

In [4]:
class MovieLensSQLLoader:
    
    def __init__(self, data_folder, file, dbhost, dbuser, dbpass, dbname, dbtable, window=10, dtype={}, date_fields=[]):
        """
        
        """
        self.data_folder = data_folder
        self.file = file
        self.engine = create_engine(f'postgresql://{dbuser}:{dbpass}@{dbhost}:5432/{dbname}')
        self.dbtable = dbtable
        self.window = window
        self.current_window = 0
        self.num_registers = 0
        self.df = pd.read_csv(f"{self.data_folder}/{self.file}",dtype=dtype, parse_dates=date_fields)
    
    def load(self, delay=0, if_exists='append', restart=False):
        """
        
        """
        self.current_window = 0 if restart else self.current_window
        num_iters = math.ceil(len(self.df)/self.window)
        for i in tqdm(range(num_iters)):
            if i > self.current_window:
                temp_df = self.df.iloc[i*self.window:(i+1)*self.window]
                temp_df.to_sql(
                    name=self.dbtable,
                    con=self.engine,
                    if_exists=if_exists
                )
                self.num_registers += len(temp_df)
                self.current_window = i
                time.sleep(delay)
    
    def status(self):
        return {
            "CurrentWindow":self.current_window,
            "RegistersInSQL":self.num_registers,
            "Window":self.window
        }
        
    def drop_table(self):
        """
        
        """
        self.engine.execute(f'DROP TABLE {self.dbtable}')
        # pd.DataFrame([]).to_sql(name=self.dbtable, con=self.engine, if_exists="replace")

## 1. Read data from S3

In [5]:
mloader = MovieLensSQLLoader(
    data_folder,
    file,
    dbhost,
    dbuser,
    dbpass,
    dbname,
    window = 10,
    dbtable='tags',
    dtype = {
        "userId":np.int64,
        "movieId":np.int64,
        "tag":np.str,
    },
    date_fields = ["timestamp"]
)
display(mloader.df.dtypes)
mloader.df.head(3)

userId        int64
movieId       int64
tag          object
timestamp    object
dtype: object

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598


## 2. Loading data do RDS (Postgres)

In [6]:
mloader.load()

  0%|          | 14/109336 [00:25<56:00:21,  1.84s/it]


KeyboardInterrupt: 

In [ ]:
mloader.status()

In [ ]:
#mloader.drop_table()

-----

## Anexos

Execute the following command if you need to install **pgdb(postgres)** in the notebook's environment
```python 
! pip install pgdb
```

In [ ]:
import pgdb

In [ ]:
myConnection = pgdb.Connection( host=dbhost, user=dbuser, password=dbpass, database=dbname)
def query( conn ) :
    cur = conn.cursor()
    cur.execute( "SELECT * FROM tags LIMIT 3" )
    for a, b, *otros in cur.fetchall() :
        print( a, b )
query( myConnection )
myConnection.close()